# Create Slice for Fabric Rack
This is the slice creation for the ```Ansible``` tutorial, This is the first step of the assignment, to create the slice on fabric using fablib
 

## 1. Set up the Experiment

In this section you will use the Fablib manager to create a new slice that is composed of 3 nodes for the IPv4 Assignment.
### 1.1 Reserve Resources
In this section we will declare the structure of the slice we will be using.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "ExploringAnsible"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 4
ram = 16
disk = 50

# Create a slice with a Server node and two Client nodes
try:
    #Create Slice
    slice = fablib.new_slice(slice_name)
    
    # Add node
    node1 = slice.add_node(name='client', site=site)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    PortCS = node1.add_component(model='NIC_Basic', name='CL-P').get_interfaces()[0]
    PortCA = node1.add_component(model='NIC_Basic', name='CL-A').get_interfaces()[0]
    
    # Add node
    node2 = slice.add_node(name='server', site=site)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    PortSC = node2.add_component(model='NIC_Basic', name='SV-P').get_interfaces()[0] 
    PortSA = node2.add_component(model='NIC_Basic', name='SV-A').get_interfaces()[0] 
    
    node3 = slice.add_node(name='ansible', site=site)
    node3.set_capacities(cores=cores, ram=ram, disk=disk)
    node3.set_image(image)
    PortAS = node3.add_component(model='NIC_Basic', name='An-S').get_interfaces()[0] 
    PortAC = node3.add_component(model='NIC_Basic', name='An-C').get_interfaces()[0] 
    
    # Network
    net1 = slice.add_l2network(name='LAN_SC', interfaces=[PortCS, PortSC])
    net2 = slice.add_l2network(name='LAN_AC', interfaces=[PortCA, PortAC])
    net3 = slice.add_l2network(name='LAN_AS', interfaces=[PortSA, PortAS])

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

### 1.2 Set up the Experiment Network
This section of the slice creation, once the nodes are active, will assign the network spaces some predefined values that we will use for the Assignment.

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

CL = 'client'
SV = 'server'
AN = 'ansible'

LAN_CS = 'LAN_SC'
LAN_AS = "LAN_AS"
LAN_AC = "LAN_AC"

CL_IP = '10.10.10.2' #SR -> AN
SR_IP = '10.10.10.1' #AN -> SR


CL_AN_IP = '12.1.1.1' #SR -> AN
SR_AN_IP = '11.1.1.1' #AN -> SR
AN_CL_IP = '12.1.1.2' #SR -> AN
AN_SR_IP = '11.1.1.2' #AN -> SR

# configure nodes
try:
    CltNode = slice.get_node(CL)
    SrvNode = slice.get_node(SV)
    AnsNode = slice.get_node(AN)
    #Client
    C_iface1= CltNode.get_interface(network_name=LAN_CS) 
    C_iface1.set_ip(ip=CL_IP, cidr="24")
    C_iface1= CltNode.get_interface(network_name=LAN_AC) 
    C_iface1.set_ip(ip=CL_AN_IP, cidr="24")
    
    #Server
    S_iface2= SrvNode.get_interface(network_name=LAN_CS) 
    S_iface2.set_ip(ip=SR_IP, cidr="24")
    S_iface2= SrvNode.get_interface(network_name=LAN_AS) 
    S_iface2.set_ip(ip=SR_AN_IP, cidr="24")

    #Ansiblenode
    A_iface2= AnsNode.get_interface(network_name=LAN_AC) 
    A_iface2.set_ip(ip=AN_CL_IP, cidr="24")
    A_iface2= AnsNode.get_interface(network_name=LAN_AS) 
    A_iface2.set_ip(ip=AN_SR_IP, cidr="24")
    
    for node in slice.get_nodes():        
        for iface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev {iface.get_device_name()} up')
        stdout, stderr = node.execute(f'ip a')
        
        node.execute("sudo sysctl -w net.ipv4.ip_forward=1")
except Exception as e:
    print(f"Fail: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
In this section of the slice creation, we will add any aditional software, tools or scripts that we need for our experiments.

In [ ]:
# Install Software
import io
import paramiko

try:
    AnsNode.upload_file('src/hellogeni-install.tar.gz', 'hellogeni-install.tar.gz')
    AnsNode.execute("tar xvfz hellogeni-install.tar.gz")

except Exception as e:
    print(f"Error Uploading tar file: {e}")

try:
    
    for node in slice.get_nodes():
        node.execute('sudo apt-get update')
        node.execute('sudo apt-get upgrade -y')
        node.execute('sudo apt-get install python -y')
        node.execute('sudo apt-get install python-pip -y')
        #install omni requirements
        node.execute('sudo apt-get install swig python-openssl libxmlsec1 xmlsec1 -y\
                     libxmlsec1-openssl libxmlsec1-dev -y')
        #Note: This will set python2 as the default python version on your Ubuntu 20.04 machine.

        node.execute('curl https://bootstrap.pypa.io/pip/2.7/get-pip.py --output get-pip.py')
        node.execute('sudo python2.7 get-pip.py')
        node.execute('sudo apt-get install build-essential swig python-dev libssl-dev -y')
        node.execute('pip2 install M2Crypto==0.30.0')
        node.execute('pip2 install python-dateutil')
    
    #omni install
    AnsNode.execute('wget https://github.com/GENI-NSF/geni-tools/archive/refs/tags/v2.11.tar.gz')


    # Extract the tarball
    AnsNode.execute('tar -xvzf "v2.11.tar.gz"')
    # Move the extracted directory to the target location
    AnsNode.execute('sudo mv "./geni-tools-2.11/" "/usr/local/bin/geni-tools-2.11"')
    AnsNode.execute('cd "/usr/local/bin/"')
    AnsNode.execute('sudo ln -s geni-tools-2.11/ gcf')
    AnsNode.execute('cd "/home/ubuntu/"')

    
    # set PATH so it includes geni software if it exists
    AnsNode.upload_file('scripts/install_omni.sh','install_omni.sh')
    AnsNode.upload_file('scripts/inventory','inventory')
    AnsNode.upload_file('scripts/E2_template','E2_template')
    AnsNode.execute('chmod +x install_omni.sh')
    AnsNode.execute('source install_omni.sh')
        
    #install Ainsible on NodeS
    AnsNode.execute("sudo apt-get install pip -y")
    AnsNode.execute("sudo apt install software-properties-common -y" )
    AnsNode.execute("sudo add-apt-repository --yes --update ppa:ansible/ansible" )
    AnsNode.execute("sudo apt-get install ansible -y" )
    
except Exception as e:
    print(f"Error installing Ansible: {e}")

try:
    key_name = "Ansible_Key"
    #private_key_path = "REPLACE_WITH_PRIVATE_KEY_PATH_FOR_PORTAL_KEYS" {key.get_name()} {key.get_base64()}
    key = paramiko.RSAKey.generate(bits=2048)
    public_key_string = f"{key.get_name()} {key.get_base64()}"
    CltNode.add_public_key(sliver_public_key=public_key_string)
    SrvNode.add_public_key(sliver_public_key=public_key_string)
    x = key.get_name()

    #create a temporary pipe because geting key as a string is not valid
    tmpPipe = io.StringIO()
    key.write_private_key(tmpPipe)
    tmpPipe.seek(0)
    y = tmpPipe.read()

    
    print(f"{x} << echo '{y}'")
    
    AnsNode.execute(f"echo '{y}' >> {key_name}")
    AnsNode.execute(f"chmod 600 {key_name}")
    
except Exception as e:
    print("Error: sending keys", str(e))

## Continue to The experiment notebook

Once You have completed this notebook you should be able to continue to the Routing IPV4 Notebook. You can either open it on the explorer or click [Here](./HelloAnsible.ipynb) to open the next notebook.